In [3]:
import sklearn
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
import json
import re
import uuid
import os
import time

from goose3 import Goose
from collections import Counter

headlineRegex = re.compile(r'http.*?/world')

keyWords = [""]
#keyWords = ["measles", "disease"]
#domain = "nationalreview.com"
#domain = "dailywire.com"
#domain = "foxnews.com"
#domain = "bbc.com"
#domain = "economist.com"
#domain = "npr.org"
#domain = "slate.com"

#    "huffpost.com",
#    "nytimes.com",
#    "washingtonpost.com",
#    "nationalreview.com",
#   "dailywire.com",
#    "foxnews.com",
#    "bbc.com",
#    "cnn.com",
#    "economist.com",
#    "npr.org",
#    "slate.com"

domains = [
    "nationalreview.com",
    "foxnews.com",
    "cnn.com",
    "npr.org"
]

themes = ["EDUCATION",
"ELECTION",
"SCIENCE",
"DEMOCRACY",
"REFUGEES",
"EXTREMISM",
"TAX_FNCACT_PRESIDENTS",
"UNGP_HEALTHCARE",
"CORRUPTION",
"DRUG_TRADE",
"AGRICULTURE",
"ECON_STOCKMARKET",
"BULLYING",
"FREESPEECH",
"SOC_INNOVATION",
"HEALTH_VACCINATION",
"MOVEMENT_ENVIRONMENTAL",
"MOVEMENT_SOCIAL",
"NATURAL_DISASTER_FLOODS",
"NATURAL_DISASTER_HURRICANES",
"PROPAGANDA",
"ECON_ENTREPRENEURSHIP",
"PUBLIC_TRANSPORT",
"IDEOLOGY",
"DISCRIMINATION",
"IMMIGRATION",
"POVERTY",
"UNEMPLOYMENT",
"CYBER_ATTACK",
"SOVEREIGNTY",
"TAX_DISEASE_DISEASES",
"INSURGENCY",
"PERSECUTION",
"MEDIA_CENSORSHIP",
"TAX_FNCACT_VICE_PRESIDENT",
"MANMADE_DISASTER_PLANE_CRASH"]

baseDir = "/tmp"
baseUri = "https://api.gdeltproject.org/api/v2/doc/doc?query="
attempts = 3
maxrecords = 250
if keyWords:
    queryBase = "%20".join(keyWords)

g = Goose()

#"timespan" : "24months",
uriBuilder = {
    "mode" : "artlist",
    "maxrecords" : str(maxrecords),
    "format" : "json"
}

for newsDomain in domains:
    for newsTheme in themes:
        time.sleep(1)
        queryBuilder = {
            "sourcelang" : "english",
            "domainis": newsDomain,
            "theme": newsTheme
        }
        
        query = queryBase
        for key, val in queryBuilder.items():
            query += "%20" + key + ":" + val

        targetUri = baseUri + query
        for key, val in uriBuilder.items():
            targetUri += "&" + key + "=" + val

        attempt = 1
        while attempt <= attempts:
            try:
                print(targetUri)
                with urllib.request.urlopen(targetUri) as url:
                    cleanQuery = url.read().replace(b'\n', b'')
                    string = cleanQuery.decode('utf-8')

                break
            except (ConnectionError, HTTPError, Timeout):
                attempt += 1
                print('need to retry query')
                time.sleep(5)

        #print('need to retry query')
        #continue
        #with urllib.request.urlopen(targetUri) as url:
        #    cleanQuery = url.read().replace(b'\n', b'')
        #    string = cleanQuery.decode('utf-8')
            
        #print("query: %s" % string)
        print("data for %s %s" % (newsDomain, newsTheme))
        if string == '{}':
            #print(targetUri)
            print("no data for %s %s" % (newsDomain, newsTheme))
            continue

        artDict = json.loads(string)
        articles = artDict.get("articles")

        urls = [val for item in articles for key, val in item.items() if key=="url"]
        headlines = [val for item in articles for key, val in item.items() if key=="title"]
        seenDates = [val for item in articles for key, val in item.items() if key=="seendate"]
        
        time.sleep(1) 
        for newsUrl, newsHeadline, newsDate in zip(urls, headlines, seenDates):
            try:
                article = g.extract(url=newsUrl)
                fileData = {
                    "domain": newsDomain,
                    "theme": newsTheme,
                    "headline": newsHeadline,
                    "date": newsDate,
                    "url": newsUrl,
                    "title": article.title,
                    "article": article.cleaned_text
                }
        
                filename = newsDomain + "." + newsTheme + "." + str(uuid.uuid4()) + ".json"
                workDir = os.path.join(baseDir, newsDomain)
                if not os.path.exists(workDir):
                    os.mkdir(workDir)

                filePathName = os.path.join(workDir, filename)

                print("filename: %s" % filePathName)
                with open(filePathName, 'w') as jf:
                    json.dump(fileData, jf, indent=2)

            except Exception:
                print("skipping article: %s" % filePathName)
                time.sleep(3)
                continue
                
        
print("done")

https://api.gdeltproject.org/api/v2/doc/doc?query=%20sourcelang:english%20domainis:npr.org%20theme:EDUCATION&mode=artlist&maxrecords=250&format=json
data for npr.org EDUCATION
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.a77a85a3-093f-4f36-a453-04fddfb0ab72.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.ed9011f7-23c6-4a72-b280-3dd5807abeed.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.6d2efd5f-46e5-45c9-9f85-ba84f7c9751b.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.870ff7bd-6a66-4107-ac0a-e62a2b75990d.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.f3728159-4d10-4a33-b32c-9efe31eef6a3.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.531f8166-364b-49f8-9a70-57933f1abb88.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATIO

filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.c04f3f26-7b9a-4500-b9c3-b0047829ad80.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.ee89ad5c-301b-4dbe-9252-a58dafea40a6.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.d6eaef17-d494-4e08-8451-05d0c676c251.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.26e93085-d55a-45fa-af26-4f4939dccb4e.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.fcea4a03-df03-40c6-876d-c7a0b5d5727e.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.821b9c94-80b1-4faf-827a-a9cf2f220703.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.e3555720-a2d0-495c-b865-1f3a31184099.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.ede695ca-2f73-4a6d-89f4-5701c195c1d1.json
filename

filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.339a44e0-6cb2-439a-b778-8400caa10f44.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.a16ad300-7fdf-4842-a439-d56394754499.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.5419f91f-20c2-4ed9-99eb-dd1e3835afad.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.d57a5aad-7f70-4e11-805d-e94653d05a14.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.fa7fce4e-5f07-4f99-8269-b5901e66952f.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.8fa4f950-bcad-480f-b20f-ac4c57cb7a8a.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.ecb1b757-a199-4849-aab1-3631b1a315d6.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.9015781c-e0c6-466b-98f0-c744e04687e0.json
filename

filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.8aa90ae4-630e-4de0-ba8c-ff78da5ce4ba.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.1ae69a15-8e76-49a0-842d-cbd7b6fb31ac.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.01dd1691-1dbb-46de-b2fa-0a10761f5a21.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.eb6112db-1e0a-43bd-a066-1b4c28288a76.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.4cc996df-1c1c-4da6-81f1-f8b1f6530b8d.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.eca5de72-b17e-4164-b810-68f5e68417a5.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.bf119691-cf1f-46a9-ba59-505bb807a43e.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.EDUCATION.037d41ad-aa3e-4b8d-b69d-11dfd452a9e3.json
filename

filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.ELECTION.4675062f-e34d-48d9-a3c4-da8a8918aacb.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.ELECTION.151afe2f-cb0f-47e0-813f-a6fa9b2f9cd7.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.ELECTION.d4664e71-1558-4c15-be05-c136155378f6.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.ELECTION.42125f5f-56d8-49d9-b8e8-dc8da37bd07f.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.ELECTION.e8f3d4b8-7632-4179-837c-10698b53f6d4.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.ELECTION.ff06c6e7-78b7-4416-87ac-7aa9ee444f4c.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.ELECTION.1d553df9-410a-49da-bdff-1dbd1e9c66f1.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.ELECTION.d1340a5e-5e0f-4315-bc02-cd2a71f731b2.json
filename: /Users

...

filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_DISEASE_DISEASES.a18f4f47-1133-48ec-a067-262343de6acc.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_DISEASE_DISEASES.072a1708-c1f9-471b-a125-6a26b4141aa7.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_DISEASE_DISEASES.0d771271-e10d-4dbf-9b2e-885e743d5833.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_DISEASE_DISEASES.35765351-8a53-4333-9b6c-df1d0e6b149a.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_DISEASE_DISEASES.98113d79-8627-4dc1-a342-faab4c92db73.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_DISEASE_DISEASES.daeae91f-ebe0-4434-acd4-1e4291e65b86.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_DISEASE_DISEASES.1bafe457-f18b-49c4-9f7b-b0cd3dc1a891.json
filename: /Users/arashj/Documents/courses/cs221/project

filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.MEDIA_CENSORSHIP.17185d96-d3c8-4c41-af22-1c756a404820.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.MEDIA_CENSORSHIP.1263d714-6d63-4823-a48d-907273324dd0.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.MEDIA_CENSORSHIP.aa62bea9-be09-4326-b6eb-2c48eb48ba4c.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.MEDIA_CENSORSHIP.c095eaac-23ca-4855-9bb3-b68f70f8b97f.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.MEDIA_CENSORSHIP.66174ee5-5d82-4d6c-ae07-1485acfb6468.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.MEDIA_CENSORSHIP.149e3bcd-4ef3-40a9-b673-76d8eed4b752.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.MEDIA_CENSORSHIP.e25736d5-265d-475c-ba74-bf9ee9a12160.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.MEDIA_CENSO

filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.ff3e6cfb-6f55-46f0-a4e6-c84401e7e53c.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.df3ae6f1-0039-48d4-b10e-6a37b81b9aef.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.d0a88866-beec-42ec-8d46-b091e7d40be7.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.81f766e6-62e2-4627-800e-b03add2fb40e.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.5f64fc12-16f5-4305-91db-d66bd58dd805.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.d5a9e54d-8483-49d7-b703-377922a1296e.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.2601dc45-6d65-4567-bb34-1362777dc7d4.json
filename: /Users/ara

filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.9c81ba5a-521c-47d5-ac66-3be808376251.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.a313dd9f-8a94-4364-8226-9fafeca21bcc.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.ab870f5c-f12e-4181-8db1-d0d69de743bf.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.f1e415d4-54e3-4cf4-9f09-821f6ca99965.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.309dcbaa-7eac-4581-aed4-189c8458355a.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.de4bcb0d-f3b6-4b76-b2fc-4db929ad4f47.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.8004d5c5-72e2-4554-8f52-cc6f58dfc45c.json
filename: /Users/ara

filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.9f2ddbbb-5d54-4bc8-9bfa-6cb9ac40b039.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.9bbd122d-4dbe-4e14-994d-e90ab86575e1.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.ddbdd77f-2a60-4755-a923-00897358f124.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.c6e1687a-7a75-4abe-a626-91f841b08769.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.56eb2735-b0ad-4ddc-9d6c-23cfeaaeee65.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.75f31178-abe9-40b4-ae0d-9436ddb1072e.json
filename: /Users/arashj/Documents/courses/cs221/project/npr.org/npr.org.TAX_FNCACT_VICE_PRESIDENT.21e9ba56-14c4-4fca-bce4-44ce7f6bb57d.json
filename: /Users/ara

In [2]:
import random

# base line: Generate a headline by randomly picking sequence of words for each position in the headline.
# For each position in the headline, create a dictionary and map the possible following words
# Inspired by https://www.perlmonks.org/index.pl?node_id=94856

# Headline length is fixed 10 words
fixedLength = 10
dictList = [{} for i in range(0, fixedLength)]

for news in headlines:
    words = [word for word in news.split(" ") if word and len(word) > 1]
    words = words[0:fixedLength]
          
    for index, w in enumerate(reversed(words)):
        if index == 0:
            followingWord = w
            continue
        
        cDict = dictList[index]
        if w not in cDict:
            cDict[w] = []

        cDict[w].append(followingWord)
        followingWord = w
     
rDict = reversed(dictList[1:])
newSent = []
nextValue = None
for pos in rDict:

    if not nextValue:
        nextValue = random.sample((pos.keys()), 1).pop()

    possNextList = pos[nextValue]
    newSent.append(nextValue)
    nextValue = random.sample(possNextList, 1).pop()
        
print(" ".join(newSent))


Boy Who Wasnt Immunized For Tetanus Spent 57 Days


In [3]:
#!pip install goose3
from goose3 import Goose
g = Goose()

url = urls[0]

article = g.extract(url=url)
article.title
article.cleaned_text[:150]


'Measles Is Spiking Around The Globe. How Worried Should We Be?\n\nThe world has two kinds of measles problems.\n\nIn low-income countries like Madagascar '